# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)
print('\n' ,file_path_list[:5], '\n files count:' ,len(file_path_list))

/home/workspace

 ['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv'] 
 files count: 30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))

# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[:2])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
[['Stephen Lynch', 'Logged In', 'Jayden', 'M', '0', 'Bell', '182.85669', 'free', 'Dallas-Fort Worth-Arlington, TX', 'PUT', 'NextSong', '1.54099E+12', '829', "Jim Henson's Dead", '200', '1.54354E+12', '91'], ['Manowar', 'Logged In', 'Jacob', 'M', '0', 'Klein', '247.562', 'paid', 'Tampa-St. Petersburg-Clearwater, FL', 'PUT', 'NextSong', '1.54056E+12', '1049', 'Shell Shock', '200', '1.54354E+12', '73']]


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION =
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}"""
)
    
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

> 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
> 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
> 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

> * After examining the query, it was clear that the sessionId and itemInSession are the main columns to go into the where clause and deremine how to get the required query result. Moreover the combination of the two values provides a unique identifier for the rows. So, the primary key was set to be a **Compound key** of session_id and item_in_session. 

> * In additoin, the table name is set to **session_item** to reflect the variable involved in partioning and retrieval of the data.

> * I have considered the **ordering of the columns in the CREATE and INSERT statements** to match the order of the compound primary key columns.

> * To render the query results in a readable format with column name headings the prettytable library was used to process the query output in all the SELECT queries.

In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS session_item " # the tabl name is set to reflect the main variable used in the partioning for retrieval of the data.
query = query + "(session_id int, item_in_session int, artist text, song_title text, length float, PRIMARY KEY(session_id, item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
# Set up the CSV file. 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## Assign the INSERT statements into the `query` variable
        
        query = "INSERT INTO session_item(session_id, item_in_session, artist, song_title, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[-3]), int(line[3]), line[0], line[-2], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Add in the SELECT statement to verify the data was entered into the table

query_1 = "SELECT * FROM session_item WHERE session_id = 338  AND item_in_session = 4"
try:
    rows = session.execute(query_1)
except Exception as e:
    print(e)

t = PrettyTable(['Session_id', 'Item_in_Session', 'Artist', 'Song_title', 'Length'])
for row in rows:
    # Adding the rows from the resulting query to the table
    t.add_row([row.session_id, row.item_in_session, row.artist, row.song_title, round(row.length,2)])

print(t)    

+------------+-----------------+-----------+---------------------------------+--------+
| Session_id | Item_in_Session |   Artist  |            Song_title           | Length |
+------------+-----------------+-----------+---------------------------------+--------+
|    338     |        4        | Faithless | Music Matters (Mark Knight Dub) | 495.31 |
+------------+-----------------+-----------+---------------------------------+--------+


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182    

> * IN this query, the main determinants on which the data partioning and sorting is based are **user_id**, **session_id** and **item_in_session** variables respectively. So, the first two paramaters are set to formulate a **composite partition key** and the **item_in_session** is set to be a **clustering column** for sorting the result.

> * The table name is set to **user_session** to reflect the partioning columns used for the table.

> * I have considered the **ordering of the columns in the CREATE and INSERT statements** to match the order of the composite primary key columns.

In [11]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS user_session "
query = query + "(user_id int, session_id int, item_in_session int, artist text, song_title text, user_first_name text, user_last_name text, PRIMARY KEY((user_id, session_id), item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [12]:
# Set up the CSV file

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_session( user_id, session_id, item_in_session ,artist, song_title, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[-1]), int(line[-3]), int(line[3]), line[0], line[-2], line[1], line[4]))

**Do a SELECT to verify that the data have been inserted into each table**

In [13]:
## Add in the SELECT statement to verify the data was entered into the table and validate the next query without

query = "select * from user_session WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

    
# User PrettyTable to render aesthetically readable query results with column headings
t = PrettyTable(['Artist', 'Song_title', 'Item_in_Session', 'User_first_name', 'User_last_name', 'User_id', 'Session_id'])
for row in rows:
    # Adding the rows from the resulting query to the table
    t.add_row([row.artist, row.song_title, row.item_in_session, row.user_first_name, row.user_last_name, row.user_id, row.session_id])

print(t)

+-------------------+------------------------------------------------------+-----------------+-----------------+----------------+---------+------------+
|       Artist      |                      Song_title                      | Item_in_Session | User_first_name | User_last_name | User_id | Session_id |
+-------------------+------------------------------------------------------+-----------------+-----------------+----------------+---------+------------+
|  Down To The Bone |                  Keep On Keepin' On                  |        0        |      Sylvie     |      Cruz      |    10   |    182     |
|    Three Drives   |                     Greece 2000                      |        1        |      Sylvie     |      Cruz      |    10   |    182     |
| Sebastien Tellier |                      Kilometer                       |        2        |      Sylvie     |      Cruz      |    10   |    182     |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |      

**Here is the actual SELECT query retrieving just the needed columns**

In [14]:
##  SELECT statement for only name of artist, song (sorted by itemInSession) and user (first and last name)
## the previous query should validate the results of this query

query_2 = "SELECT artist, song_title, user_first_name, user_last_name FROM user_session WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(query_2)
except Exception as e:
    print(e)


# User PrettyTable to render aesthetically readable query results with column headings
t = PrettyTable(['Artist', 'Song_title', 'User_first_name', 'User_last_name'])
for row in rows:
    # Adding the rows from the resulting query to the table
    t.add_row([row.artist, row.song_title, row.user_first_name, row.user_last_name])

print(t)

+-------------------+------------------------------------------------------+-----------------+----------------+
|       Artist      |                      Song_title                      | User_first_name | User_last_name |
+-------------------+------------------------------------------------------+-----------------+----------------+
|  Down To The Bone |                  Keep On Keepin' On                  |      Sylvie     |      Cruz      |
|    Three Drives   |                     Greece 2000                      |      Sylvie     |      Cruz      |
| Sebastien Tellier |                      Kilometer                       |      Sylvie     |      Cruz      |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |      Sylvie     |      Cruz      |
+-------------------+------------------------------------------------------+-----------------+----------------+


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

> * IN the last query the primary key, is set to a compound key of the song_title as partion key and user_id as a clustering column. The combination of both provides the unique row identifier. 

> * song_user is used as the name for the table to emphasis the query objective of obtaining users listened to certain songs.

> * I have considered the **ordering of the columns in the CREATE and INSERT statements** to match the order of the compound primary key columns.

In [15]:
## Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS song_user "
query = query + "(song_title text, user_id int, user_first_name text, user_last_name text, PRIMARY KEY(song_title, user_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [16]:
# Set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_user(song_title, user_id , user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[-2], int(line[-1]), line[1], line[4]))

**Do a SELECT to verify that the data have been inserted into each table**

In [17]:
## Add in the SELECT statement to verify the data was entered into the table
## Due to the limited number of columns, SELECT * is used in the query

query_3 = "SELECT * FROM song_user WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query_3)
except Exception as e:
    print(e)

# User PrettyTable to render aesthetically readable query results with column headings
t = PrettyTable(['User_id', 'User_first_name', 'User_last_name', 'Song_title'])
for row in rows:
    # Adding the rows from the resulting query to the table
    t.add_row([row.user_id, row.user_first_name, row.user_last_name, row.song_title])

print(t)

+---------+-----------------+----------------+---------------------------+
| User_id | User_first_name | User_last_name |         Song_title        |
+---------+-----------------+----------------+---------------------------+
|    29   |    Jacqueline   |     Lynch      | All Hands Against His Own |
|    80   |      Tegan      |     Levine     | All Hands Against His Own |
|    95   |       Sara      |    Johnson     | All Hands Against His Own |
+---------+-----------------+----------------+---------------------------+


### Ectra step to display the output of the previous SELECT queries as Dataframes

I have found this [stackoverflow](https://stackoverflow.com/questions/41247345/python-read-cassandra-data-into-pandas) that helped in displaying the output query into a dataframe

In [18]:
from cassandra.query import dict_factory

session.row_factory = dict_factory

# defining a funcition to processes all the previous select statment queries form the created tables.

def output_dfs(query_1, query_2, query_3):
    """ The fuction takes as input 3 queries and returns each query output as a dataframe from calling the nested fuction on the 3 queries"""
    def output_df(query):
        """ nested fuction that processes the select query output into a dataframe """
        df = pd.DataFrame()
        for row in session.execute(query):
            df = df.append(pd.DataFrame(row, index=[0]))

        df = df.reset_index(drop=True)
        return df
    
    return(output_df(query_1), output_df(query_2), output_df(query_3))

df_1, df_2, df_3 = output_dfs(query_1, query_2, query_3)

In [19]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
df_1

,session_id,item_in_session,artist,length,song_title
0,338,4,Faithless,495.307312,Music Matters (Mark Knight Dub)


In [20]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

df_2

,artist,song_title,user_first_name,user_last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


In [21]:
## Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

df_3

,song_title,user_id,user_first_name,user_last_name
0,All Hands Against His Own,29,Jacqueline,Lynch
1,All Hands Against His Own,80,Tegan,Levine
2,All Hands Against His Own,95,Sara,Johnson


### Drop the tables before closing out the sessions

In [22]:
## Drop the table before closing out the sessions

query = "drop table session_item"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table song_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()